In [ ]:
import cv2
import glob
import matplotlib.pyplot as plt
import cv2.aruco as aruco

# https://docs.opencv.org/4.7.0/df/d4a/tutorial_charuco_detection.html - jest do niczego
# opencv 4.7.0 - wszystko na nim robię

In [ ]:
images = sorted(glob.glob('./chessboard_photos_for_calibration/nie_dziala/*.jpg'))
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_250)

img_points = []
corners_all = []
good_images = []
for iname in images:
    img = cv2.imread(iname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    board = aruco.CharucoBoard((6, 5), 0.0184, 0.015, dictionary)

    #parameters = aruco.DetectorParameters()
    detector = aruco.CharucoDetector(board)
    charuco_corners, charuco_ids, marker_corners, marker_ids = detector.detectBoard(
            image=gray)

    img = aruco.drawDetectedMarkers(
            image=img, 
            corners=marker_corners)
    img = aruco.drawDetectedCornersCharuco(
            image=img,
            charucoCorners=charuco_corners,
            charucoIds=charuco_ids)
    if len(charuco_corners) == 20:
        good_images.append(iname)
        img_points.append(charuco_corners)
        assert tuple(charuco_ids) == tuple([[i] for i in range(20)])
        corners_all.append(board.getChessboardCorners())

In [ ]:
calibration, cameraMatrix, distCoeffs, rvecs, tvecs = cv2.calibrateCamera(
        objectPoints=corners_all,
        imagePoints=img_points,
        imageSize=(1280,720),
        cameraMatrix=None,
        distCoeffs=None)


In [ ]:
print(calibration)
print(cameraMatrix)
print(distCoeffs)
#print(rvecs)
#print(tvecs)

In [ ]:
points3D = board.getChessboardCorners() # coordinates of the chessboard corners in the chessboard coordinate system
points3Da = points3D + [[0, 0, 0.01]]   # like points3D, but with a point 1cm under the chessboard
for i in range(len(good_images)):
    tvec = tvecs[i]
    rvec = rvecs[i]
    img = cv2.imread(good_images[i])
    img = cv2.drawFrameAxes(img, cameraMatrix, distCoeffs, rvec, tvec, 0.1) # draw the chessboard coordinate system
    
    # draw green corners on the chessboard
    p = cv2.projectPoints(points3D, rvec, tvec, cameraMatrix, distCoeffs)[0]
    for x, y in p[:,0,:]:
        img = cv2.circle(img, (round(x), round(y)), radius=5, color=(0, 255, 0), thickness=-1)
    
    # draw red corners under the chessboard
    p = cv2.projectPoints(points3Da, rvec, tvec, cameraMatrix, distCoeffs)[0]
    for x, y in p[:,0,:]:
        img = cv2.circle(img, (round(x), round(y)), radius=5, color=(255, 0, 0), thickness=-1)
    
    img = cv2.undistort(img, cameraMatrix, distCoeffs, None, cameraMatrix) # undistort the image (get's crazy)
    
    plt.imshow(img)
    plt.show()